In [5]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK data
nltk.download('punkt')

# Load the test data CSV
test_data_path = 'TEST DATA.csv'
test_data = pd.read_csv(test_data_path)

# Load the dictionary Excel file
dictionary_path = 'Dictionary 10.xlsx'
dictionary_data = pd.read_excel(dictionary_path)

# Initialize the stemmer
stemmer = PorterStemmer()

# Prepare a dictionary for quick lookup from dictionary data and apply stemming
category_words = {category: set(dictionary_data[category].dropna().str.lower()) for category in dictionary_data.columns}

category_words_stemmed = {}
for category, words in category_words.items():
    stemmed_words = {stemmer.stem(word) for word in words}
    category_words_stemmed[category] = stemmed_words

# Aggregate text by Text-ID
aggregated_text = test_data.groupby('Text-ID')['Text'].apply(' '.join).reset_index()

# Tokenization function
def tokenize(text):
    return word_tokenize(text.lower())

# Stemming function
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Apply tokenization and stemming to the aggregated text
aggregated_text['Tokenized'] = aggregated_text['Text'].apply(tokenize)
aggregated_text['Stemmed'] = aggregated_text['Tokenized'].apply(stem_tokens)

# Initialize a dataframe to hold the results
result_counts = pd.DataFrame(columns=['Text-ID'] + list(category_words.keys()))
result_counts['Text-ID'] = aggregated_text['Text-ID']
result_counts = result_counts.fillna(0)

# Function to count words from a set in a given tokenized and stemmed text
def count_words_in_text(tokens, word_set):
    return sum(1 for word in tokens if word in word_set)

# Iterate over the aggregated text data and count occurrences for each category
for _, row in aggregated_text.iterrows():
    text_id = row['Text-ID']
    tokens = row['Stemmed']
    
    for category, words in category_words_stemmed.items():
        count = count_words_in_text(tokens, words)
        result_counts.loc[result_counts['Text-ID'] == text_id, category] += count

# Create a binary DataFrame with the same structure as result_counts
binary_counts = result_counts.copy()
binary_counts.iloc[:, 1:] = 0  # Set all counts to 0 initially

# Iterate over each row to set the highest value to 1
for index, row in result_counts.iterrows():
    highest_category = row[1:].idxmax()
    binary_counts.at[index, highest_category] = 1

# Save the result_counts and binary_counts to Excel files
output_path_counts = 'Word_stem_Counts.xlsx'
output_path_binary = 'Word_stem_Binary.xlsx'

with pd.ExcelWriter(output_path_counts) as writer:
    result_counts.to_excel(writer, index=False, sheet_name='Counts')

with pd.ExcelWriter(output_path_binary) as writer:
    binary_counts.to_excel(writer, index=False, sheet_name='Binary')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import pandas as pd

# Load the provided Excel files
expert_binary_path = 'Expert_binary.xlsx'
word_binary_path = 'Word_stem_Binary.xlsx'

expert_binary = pd.read_excel(expert_binary_path)
word_binary = pd.read_excel(word_binary_path)

# Merge the two dataframes on 'Text-ID'
merged_df = pd.merge(expert_binary, word_binary, on='Text-ID', suffixes=('_expert', '_word'))

# Create new columns
merged_df['Expert Category'] = merged_df.filter(regex='_expert$').idxmax(axis=1).str.replace('_expert', '')
merged_df['Word Category'] = merged_df.filter(regex='_word$').idxmax(axis=1).str.replace('_word', '')
merged_df['Match'] = merged_df['Expert Category'] == merged_df['Word Category']

# Adjust the calculation of match percentage to compare categories rather than words
def calculate_category_match_percentage(expert_category, word_category):
    return 100.0 if expert_category == word_category else 0.0

# Calculate Expert Match % and Word Match %
merged_df['Expert Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Expert Category'], row['Word Category']), axis=1)
merged_df['Word Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Word Category'], row['Expert Category']), axis=1)

# Select required columns with correct suffixes
expert_columns = [col for col in merged_df.columns if col.endswith('_expert')]
word_columns = [col for col in merged_df.columns if col.endswith('_word')]
category_columns = [col.replace('_expert', '') for col in expert_columns]

# Prepare the final dataframe
final_df = merged_df[['Text-ID'] + expert_columns + word_columns + ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']]

# Rename the columns for clarity
final_df.columns = ['Text-ID'] + [f'Expert_{col.replace("_expert", "")}' for col in expert_columns] + \
                   [f'Word_{col.replace("_word", "")}' for col in word_columns] + \
                   ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']

# Save the final dataframe to a new Excel file
output_path_final = 'Final_stem_Output.xlsx'
final_df.to_excel(output_path_final, index=False)

print(f"Final output saved to {output_path_final}")


Final output saved to Final_stem_Output.xlsx


In [7]:
import pandas as pd

# Load the provided Excel files
expert_binary_path = 'Expert_binary.xlsx'
word_binary_path = 'Word_stem_Binary.xlsx'

expert_binary = pd.read_excel(expert_binary_path)
word_binary = pd.read_excel(word_binary_path)

# Merge the two dataframes on 'Text-ID'
merged_df = pd.merge(expert_binary, word_binary, on='Text-ID', suffixes=('_expert', '_word'))

# Create new columns
merged_df['Expert Category'] = merged_df.filter(regex='_expert$').idxmax(axis=1).str.replace('_expert', '')
merged_df['Word Category'] = merged_df.filter(regex='_word$').idxmax(axis=1).str.replace('_word', '')
merged_df['Match'] = merged_df['Expert Category'] == merged_df['Word Category']

# Adjust the calculation of match percentage to compare categories rather than words
def calculate_category_match_percentage(expert_category, word_category):
    return 100.0 if expert_category == word_category else 0.0

# Calculate Expert Match % and Word Match %
merged_df['Expert Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Expert Category'], row['Word Category']), axis=1)
merged_df['Word Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Word Category'], row['Expert Category']), axis=1)

# Select required columns with correct suffixes
expert_columns = [col for col in merged_df.columns if col.endswith('_expert')]
word_columns = [col for col in merged_df.columns if col.endswith('_word')]
category_columns = [col.replace('_expert', '') for col in expert_columns]

# Prepare the final dataframe
final_df = merged_df[['Text-ID'] + expert_columns + word_columns + ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']]

# Rename the columns for clarity
final_df.columns = ['Text-ID'] + [f'Expert_{col.replace("_expert", "")}' for col in expert_columns] + \
                   [f'Word_{col.replace("_word", "")}' for col in word_columns] + \
                   ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']

# Calculate total True, False, and Match %
total_true = final_df['Match'].sum()
total_false = len(final_df) - total_true
match_percentage = (total_true / len(final_df)) * 100

# Print out the totals and match percentage
print(f"Total True: {total_true}")
print(f"Total False: {total_false}")
print(f"Overall Match Percentage: {match_percentage:.2f}%")

# Save the final dataframe to a new Excel file
output_path_final = 'Final_stem_Output.xlsx'
final_df.to_excel(output_path_final, index=False)

print(f"Final output saved to {output_path_final}")


Total True: 543
Total False: 1583
Overall Match Percentage: 25.54%
Final output saved to Final_stem_Output.xlsx
